In [ ]:
'''
This Notebook was create only for learning Apache Spark.
In this notebook I modified code from reference to learn data science project in spark. 
Since we used different version of spark, I have tried my best to modify but the problem is .count function is cannot be used to result due to some problem that we cannot clarify.

Ref:
[1] Spark for Data Science:Bikramaditya Singhal, Srinivas Duvvuri

'''



In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [2]:
import os
memory = '10g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [3]:
conf = SparkConf();
sc = SparkContext(appName="put_it_all_AppName", conf =conf)

In [4]:
spark = SparkSession.builder\
                    .appName('strating spark')\
                    .getOrCreate()

In [5]:
init_data = spark.read.csv("C:/Users/New/Downloads/Data science in Action/Data set for spark/Spark-for-Data-Science-master/B04897_Spark for Data Science_Chapter 10_Code_Spark2.0_V1//Oscars.txt",
                           sep="\t",
                           header=True)

In [6]:
init_data.show(5)

+---------+------------------+-------------+--------------+-------------+-------------+--------------------+---------------+
| _unit_id|        birthplace|date_of_birth|race_ethnicity|year_of_award|        award|               movie|         person|
+---------+------------------+-------------+--------------+-------------+-------------+--------------------+---------------+
|670454353| Chisinau, Moldova|  30-Sep-1895|         White|         1927|Best Director| Two Arabian Knights|Lewis Milestone|
|670454354| Glasgow, Scotland|   2-Feb-1886|         White|         1930|Best Director|     The Divine Lady|    Frank Lloyd|
|670454355| Chisinau, Moldova|  30-Sep-1895|         White|         1931|Best Director|All Quiet on the ...|Lewis Milestone|
|670454356|       Chicago, Il|  23-Feb-1899|         White|         1932|Best Director|              Skippy|  Norman Taurog|
|670454357|Salt Lake City, Ut|  23-Apr-1894|         White|         1933|Best Director|            Bad Girl|  Frank Borzage|


In [7]:
awards = init_data.select("birthplace", "date_of_birth","race_ethnicity","year_of_award","award")\
                  .toDF("birthplace","date_of_birth","race","award_year","award")

In [8]:
awards.show(10, truncate=False)

+-------------------------+-------------+-----+----------+-------------+
|birthplace               |date_of_birth|race |award_year|award        |
+-------------------------+-------------+-----+----------+-------------+
|Chisinau, Moldova        |30-Sep-1895  |White|1927      |Best Director|
|Glasgow, Scotland        |2-Feb-1886   |White|1930      |Best Director|
|Chisinau, Moldova        |30-Sep-1895  |White|1931      |Best Director|
|Chicago, Il              |23-Feb-1899  |White|1932      |Best Director|
|Salt Lake City, Ut       |23-Apr-1894  |White|1933      |Best Director|
|Glasgow, Scotland        |2-Feb-1886   |White|1934      |Best Director|
|Bisacquino, Sicily, Italy|18-May-1897  |White|1935      |Best Director|
|Cape Elizabeth, Me       |1-Feb-1894   |White|1936      |Best Director|
|Bisacquino, Sicily, Italy|18-May-1897  |White|1937      |Best Director|
|Los Angeles, Ca          |3-Oct-1898   |White|1938      |Best Director|
+-------------------------+-------------+-----+----

In [9]:
# register temporary view of this dataset
awards.createOrReplaceTempView("award_SQL")

In [10]:
# Explore data
awards.select("award").distinct().show(10, False)

+-----------------------+
|award                  |
+-----------------------+
|Best Supporting Actress|
|Best Director          |
|Best Actress           |
|Best Actor             |
|Best Supporting Actor  |
+-----------------------+



In [11]:
# Check DOB quality
# SQL Mode
spark.sql("SELECT distinct(length(date_of_birth)) FROM award_SQL ").show()

+---------------------+
|length(date_of_birth)|
+---------------------+
|                   15|
|                    9|
|                    4|
|                    8|
|                   10|
|                   11|
+---------------------+



In [12]:
# Look at the value with unexpected length 4. Note that length varies based on month name
spark.sql("SELECT date_of_birth FROM award_SQL WHERE length(date_of_birth) = 4").show()

# This is an invalid date. We can either drop this record or give some meaningful value like 01-01-1972

+-------------+
|date_of_birth|
+-------------+
|         1972|
+-------------+



In [13]:
def fncleanDate(s):
    cleanedDate = s
    dateArray = s.split("-")
    try:
        yr = int(dateArray[2])
        if (yr < 100):
            yr = yr + 1900
            cleanedDate = "{0}-{1}-{2}".format(int(dateArray[0]), dateArray[1], yr)
    except: None
    return cleanedDate


def fncleanBirthplace(s):
    strArray = s.split(",")
    if (s == "New York City"):
        s += " USA"  # //Append USA
        cleanedBirthplace = s
    #//Append country if last element length is 2
    elif (len(strArray[len(strArray)-1])-1) == 2:
        s += " USA"
        cleanedBirthplace = "".join(s)
    else: cleanedBirthplace = s
    return cleanedBirthplace

#regist UDF
spark.udf.register("cleanDateUDF",fncleanDate)
spark.udf.register("cleanBirthplaceUDF",fncleanBirthplace)

<function __main__.fncleanBirthplace(s)>

In [14]:
from pyspark.sql.functions import substring_index

cleaned_df = spark.sql (""" 
            SELECT  cleanDateUDF(date_of_birth) AS dob,
                    cleanBirthplaceUDF(birthplace) AS birthplace,
                    substring_index(cleanBirthplaceUDF(birthplace),' ',-1) AS country,
                    (award_year - substring_index(cleanDateUDF(date_of_birth),'-',-1)) AS age, 
                    race, 
                    award
            
            FROM award_SQL
            """)

cleaned_df.show(5)

+-----------+--------------------+--------+----+-----+-------------+
|        dob|          birthplace| country| age| race|        award|
+-----------+--------------------+--------+----+-----+-------------+
|30-Sep-1895|   Chisinau, Moldova| Moldova|32.0|White|Best Director|
| 2-Feb-1886|   Glasgow, Scotland|Scotland|44.0|White|Best Director|
|30-Sep-1895|   Chisinau, Moldova| Moldova|36.0|White|Best Director|
|23-Feb-1899|     Chicago, Il USA|     USA|33.0|White|Best Director|
|23-Apr-1894|Salt Lake City, U...|     USA|39.0|White|Best Director|
+-----------+--------------------+--------+----+-----+-------------+
only showing top 5 rows



In [15]:
# spark.sql("SELECT cleanDateUDF (date_of_birth) AS dob FROM award_SQL").show(5)
# spark.sql("SELECT cleanBirthplaceUDF(birthplace) AS birthplace FROM award_SQL").show(5, truncate = False)
# spark.sql("SELECT (award_year - substring_index(cleanDateUDF(date_of_birth),'-',-1)) AS age FROM award_SQL").show(5)
# spark.sql("SELECT substring_index(cleanBirthplaceUDF(birthplace),' ',-1) AS country FROM award_SQL").show(5)


In [16]:
cleaned_df.na.drop

cleaned_df.groupBy("award","country")\
          .count()\
          .sort("country","award","count")\
          .show(5,False)

+-----------------------+---------+-----+
|award                  |country  |count|
+-----------------------+---------+-----+
|Best Actress           |Africa   |1    |
|Best Actor             |Australia|1    |
|Best Actress           |Australia|1    |
|Best Supporting Actor  |Australia|1    |
|Best Supporting Actress|Australia|1    |
+-----------------------+---------+-----+
only showing top 5 rows



In [17]:
# Re-register
cleaned_df.createOrReplaceTempView("award_SQL")

In [18]:
spark.sql("SELECT count(distinct country) country_count, count(distinct race) race_count, count(distinct award) award_count from award_SQL").show()

+-------------+----------+-----------+
|country_count|race_count|award_count|
+-------------+----------+-----------+
|           35|         6|          5|
+-------------+----------+-----------+



In [19]:
df_main = cleaned_df.select("country")\
                    .groupBy("country")\
                    .count()\
                    .withColumnRenamed("count", "freq")

df_main = df_main.orderBy("count", ascending = False)\
                 .filter(df_main.freq > 6)

# #  based on book 
# top_countries = spark.sql("""
#                              SELECT country, count(*) freq 
#                              FROM award_SQL 
#                              GROUP BY country 
#                              ORDER BY freq DESC 
#                              LIMIT 6
#                          """)
# top_countries.show()

In [20]:
import numpy as np
RDD_list = df_main.select(df_main.columns[:1]).collect() # RDD List is notreal  Python List
np_list = np.array(RDD_list) 
py_list = np_list.tolist()

print(RDD_list)
print("")
print(np_list)
print("")
print(py_list)
# npp = np.array().ะนสรหะ
# npp.tolist()

[Row(country='USA'), Row(country='England'), Row(country='France'), Row(country='Canada'), Row(country='Italy'), Row(country='Austria')]

[['USA']
 ['England']
 ['France']
 ['Canada']
 ['Italy']
 ['Austria']]

[['USA'], ['England'], ['France'], ['Canada'], ['Italy'], ['Austria']]


In [21]:
top_countries = [x for row in py_list for x in row]
# for each row in py_list 
#     for x in each row
# return to x

top_countries

['USA', 'England', 'France', 'Canada', 'Italy', 'Austria']

In [22]:
setCountry = udf(lambda s: s if s in top_countries else "Others", StringType())
cleaned_df = cleaned_df.withColumn("country", setCountry(cleaned_df.country))
cleaned_df.show(10, truncate = False)

+-----------+-------------------------+-------+----+-----+-------------+
|dob        |birthplace               |country|age |race |award        |
+-----------+-------------------------+-------+----+-----+-------------+
|30-Sep-1895|Chisinau, Moldova        |Others |32.0|White|Best Director|
|2-Feb-1886 |Glasgow, Scotland        |Others |44.0|White|Best Director|
|30-Sep-1895|Chisinau, Moldova        |Others |36.0|White|Best Director|
|23-Feb-1899|Chicago, Il USA          |USA    |33.0|White|Best Director|
|23-Apr-1894|Salt Lake City, Ut USA   |USA    |39.0|White|Best Director|
|2-Feb-1886 |Glasgow, Scotland        |Others |48.0|White|Best Director|
|18-May-1897|Bisacquino, Sicily, Italy|Italy  |38.0|White|Best Director|
|1-Feb-1894 |Cape Elizabeth, Me USA   |USA    |42.0|White|Best Director|
|18-May-1897|Bisacquino, Sicily, Italy|Italy  |40.0|White|Best Director|
|3-Oct-1898 |Los Angeles, Ca USA      |USA    |40.0|White|Best Director|
+-----------+-------------------------+-------+----

In [23]:
from pyspark.ml.feature import StringIndexer, Bucketizer, VectorAssembler
from pyspark.ml import Pipeline

# Pipeline
''' [raceIdxer] => 
        [awardIdxer] => 
            [countryIdxer] => 
                [bucketizer] => 
                    [TrainTest_split] => 
                        [VectorAssembler]'''

# calculate the frequency of String in column the rank them; 0 is the most frequent words
raceIdxer = StringIndexer(inputCol= "race", outputCol="raceIdx")
awardIdxer = StringIndexer(inputCol = "award", outputCol="awardIdx") 
countryIdxer = StringIndexer(inputCol = "country", outputCol = "countryIdx")

splits = [-float("inf"), 35.0, 45.0, 55.0, float("inf")] # Range finder, create interval (bucket) to group the data into 4 group
'''               |___1___||__2__||_3_||__4___| ''' # Visualize bucket
bucketizer = Bucketizer(splits = splits, inputCol = "age", outputCol = "age_buckets")

In [25]:
# Define data preparation pipeline
dp_pipeline = Pipeline(stages = [raceIdxer, awardIdxer, countryIdxer, bucketizer])
cleaned_df = dp_pipeline.fit(cleaned_df)\
                        .transform(cleaned_df)

In [27]:
cleaned_df.createOrReplaceTempView("cleaned_df_SQL") # turn SQL mode ON
TrainTest_split = spark.sql("SELECT * FROM cleaned_df_SQL").randomSplit([0.7, 0.3])# trainData, testData = cleaned_df1.randomSplit([0.7, 0.3])

In [28]:
print(TrainTest_split)  # split_data consist of 2 DF. We will assign them as trainData, testData later. 
trainData = TrainTest_split[0]
testData = TrainTest_split[1]
trainData.describe().show(truncate = False)
testData.describe().show(truncate = False)

[DataFrame[dob: string, birthplace: string, country: string, age: double, race: string, award: string, raceIdx: double, awardIdx: double, countryIdx: double, age_buckets: double], DataFrame[dob: string, birthplace: string, country: string, age: double, race: string, award: string, raceIdx: double, awardIdx: double, countryIdx: double, age_buckets: double]]
+-------+----------+-----------------+-------+-----------------+-----+-----------------------+-------------------+------------------+------------------+------------------+
|summary|dob       |birthplace       |country|age              |race |award                  |raceIdx            |awardIdx          |countryIdx        |age_buckets       |
+-------+----------+-----------------+-------+-----------------+-----+-----------------------+-------------------+------------------+------------------+------------------+
|count  |309       |309              |309    |309              |309  |309                    |309                |309        

In [29]:
#  raceIdx, age_buckets,countryIdx features via VectorAssembler
assembler = VectorAssembler(inputCols = ["raceIdx", "age_buckets","countryIdx"], outputCol = "features")

In [31]:
trainData = assembler.transform(trainData)
testData = assembler.transform(testData)

In [32]:
trainData.show(5)
testData.show(5)

+----------+--------------------+-------+----+-----+-------------+-------+--------+----------+-----------+-------------+
|       dob|          birthplace|country| age| race|        award|raceIdx|awardIdx|countryIdx|age_buckets|     features|
+----------+--------------------+-------+----+-----+-------------+-------+--------+----------+-----------+-------------+
|1-Apr-1885| Kansas City, Mo USA|    USA|47.0|White|   Best Actor|    0.0|     2.0|       0.0|        2.0|[0.0,2.0,0.0]|
|1-Dec-1935|    Brooklyn, Ny USA|    USA|43.0|White|Best Director|    0.0|     1.0|       0.0|        1.0|[0.0,1.0,0.0]|
|1-Feb-1894|Cape Elizabeth, M...|    USA|42.0|White|Best Director|    0.0|     1.0|       0.0|        1.0|[0.0,1.0,0.0]|
|1-Feb-1894|Cape Elizabeth, M...|    USA|47.0|White|Best Director|    0.0|     1.0|       0.0|        2.0|[0.0,2.0,0.0]|
|1-Feb-1894|Cape Elizabeth, M...|    USA|48.0|White|Best Director|    0.0|     1.0|       0.0|        2.0|[0.0,2.0,0.0]|
+----------+--------------------

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.feature import StringIndexer, IndexToString, VectorIndexer
from pyspark.ml.classification import LogisticRegression, OneVsRest

In [34]:
dtreeModel = DecisionTreeClassifier(labelCol = "awardIdx", featuresCol = "features").fit(trainData)
dtree_predictions = dtreeModel.transform(testData)

In [35]:
dtree_predictions.show(5)

+----------+--------------------+-------+----+-----+-------------+-------+--------+----------+-----------+-------------+--------------------+--------------------+----------+
|       dob|          birthplace|country| age| race|        award|raceIdx|awardIdx|countryIdx|age_buckets|     features|       rawPrediction|         probability|prediction|
+----------+--------------------+-------+----+-----+-------------+-------+--------+----------+-----------+-------------+--------------------+--------------------+----------+
|1-Aug-1965|Reading, Berkshir...|England|35.0|White|Best Director|    0.0|     1.0|       2.0|        1.0|[0.0,1.0,2.0]|[7.0,3.0,2.0,3.0,...|[0.46666666666666...|       0.0|
|1-Feb-1901|       Cadiz, Oh USA|    USA|34.0|White|   Best Actor|    0.0|     2.0|       0.0|        0.0|    (3,[],[])|[39.0,0.0,4.0,20....|[0.56521739130434...|       0.0|
|1-Jul-1899|Victoria Hotel, S...|England|35.0|White|   Best Actor|    0.0|     2.0|       2.0|        1.0|[0.0,1.0,2.0]|[7.0,3.0,2

In [36]:
result_DT_full = dtree_predictions.filter(dtree_predictions["awardIdx"] != dtree_predictions["prediction"])
result_DT_full.show(5)

+----------+--------------------+-------+----+-----+--------------------+-------+--------+----------+-----------+-------------+--------------------+--------------------+----------+
|       dob|          birthplace|country| age| race|               award|raceIdx|awardIdx|countryIdx|age_buckets|     features|       rawPrediction|         probability|prediction|
+----------+--------------------+-------+----+-----+--------------------+-------+--------+----------+-----------+-------------+--------------------+--------------------+----------+
|1-Aug-1965|Reading, Berkshir...|England|35.0|White|       Best Director|    0.0|     1.0|       2.0|        1.0|[0.0,1.0,2.0]|[7.0,3.0,2.0,3.0,...|[0.46666666666666...|       0.0|
|1-Feb-1901|       Cadiz, Oh USA|    USA|34.0|White|          Best Actor|    0.0|     2.0|       0.0|        0.0|    (3,[],[])|[39.0,0.0,4.0,20....|[0.56521739130434...|       0.0|
|1-Jul-1899|Victoria Hotel, S...|England|35.0|White|          Best Actor|    0.0|     2.0|     

In [38]:
result_DT = dtree_predictions.select("award","awardIdx","prediction")\
                             .filter(dtree_predictions.awardIdx != dtree_predictions.prediction)

result_DT.show(5)

+--------------------+--------+----------+
|               award|awardIdx|prediction|
+--------------------+--------+----------+
|       Best Director|     1.0|       0.0|
|          Best Actor|     2.0|       0.0|
|          Best Actor|     2.0|       0.0|
|       Best Director|     1.0|       4.0|
|Best Supporting A...|     4.0|       2.0|
+--------------------+--------+----------+
only showing top 5 rows



In [39]:
RFmodel = RandomForestClassifier(labelCol = "awardIdx", featuresCol = "features", numTrees =6).fit(trainData)
RF_predictions = RFmodel.transform(testData)

In [40]:
RF_predictions.filter(RF_predictions["awardIdx"] != RF_predictions["prediction"]).show(5)

+----------+--------------------+-------+----+-----+--------------------+-------+--------+----------+-----------+-------------+--------------------+--------------------+----------+
|       dob|          birthplace|country| age| race|               award|raceIdx|awardIdx|countryIdx|age_buckets|     features|       rawPrediction|         probability|prediction|
+----------+--------------------+-------+----+-----+--------------------+-------+--------+----------+-----------+-------------+--------------------+--------------------+----------+
|1-Aug-1965|Reading, Berkshir...|England|35.0|White|       Best Director|    0.0|     1.0|       2.0|        1.0|[0.0,1.0,2.0]|[2.10069603272928...|[0.35011600545488...|       0.0|
|1-Feb-1901|       Cadiz, Oh USA|    USA|34.0|White|          Best Actor|    0.0|     2.0|       0.0|        0.0|    (3,[],[])|[3.42929945768377...|[0.57154990961396...|       0.0|
|1-Jul-1899|Victoria Hotel, S...|England|35.0|White|          Best Actor|    0.0|     2.0|     

In [41]:
RF_predictions.select("award","awardIdx","prediction")\
              .filter(RF_predictions["awardIdx"] != RF_predictions["prediction"]).show(5)

+--------------------+--------+----------+
|               award|awardIdx|prediction|
+--------------------+--------+----------+
|       Best Director|     1.0|       0.0|
|          Best Actor|     2.0|       0.0|
|          Best Actor|     2.0|       0.0|
|       Best Director|     1.0|       4.0|
|Best Supporting A...|     4.0|       2.0|
+--------------------+--------+----------+
only showing top 5 rows



In [42]:
classifier = LogisticRegression(labelCol = "awardIdx", featuresCol="features", maxIter = 30, tol = (1E-6), fitIntercept = True)

In [43]:
ovrModel = OneVsRest(classifier = classifier, labelCol = "awardIdx", featuresCol = "features").fit(trainData)
OVR_predictions = ovrModel.transform(testData)

In [50]:
OVR_predictions.show()
"""????????????????????????????????????????????????"""

Py4JJavaError: An error occurred while calling o2047.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 172.0 failed 1 times, most recent failure: Lost task 0.0 in stage 172.0 (TID 1352, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<raceIdx:double,age_buckets:double,countryIdx:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.

In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [46]:
f1_eval = MulticlassClassificationEvaluator(labelCol="awardIdx") #Default metric is F1

In [47]:
# WeightedPrecision
wp_eval = MulticlassClassificationEvaluator(labelCol="awardIdx", metricName="weightedPrecision")

In [48]:
# Accuracy
acc_eval = MulticlassClassificationEvaluator(labelCol="awardIdx", metricName="Accuracy")

In [49]:
# Compute measures for all models
f1_eval_list = [ f1_eval.evaluate(x) for x in [dtree_predictions, RF_predictions, OVR_predictions]]
wp_eval_list = [ wp_eval.evaluate(x) for x in [dtree_predictions, RF_predictions, OVR_predictions]]
wr_eval_list = [ wr_eval.evaluate(x) for x in [dtree_predictions, RF_predictions, OVR_predictions]]

Py4JJavaError: An error occurred while calling o2092.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 170.0 failed 1 times, most recent failure: Lost task 0.0 in stage 170.0 (TID 1351, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<raceIdx:double,age_buckets:double,countryIdx:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext

In [ ]:
# '''test: cleanDateUDF(date_of_birth) AS dob'''
# s = "30-Sep-95"
# dateArray = s.split("-")
# print(dateArray)
# print(dateArray[2])

In [ ]:
# yr = int(dateArray[2])
# if (yr < 100):
#     yr = yr + 1900 # make it 4 digit
#     cleanedDate = "{0}-{1}-{2}".format(int(dateArray[0]), dateArray[1], yr)
#     print(cleanedDate)

In [ ]:
# """cleanBirthplaceUDF(birthplace)"""
# # s = "Chisinau, Moldova"
# s = "Los Angeles, Ca"
# cleanedBirthplace = "cleanedBirthplace"
# strArray = s.split(",")
# strArray

In [ ]:
# if (s == "New York City"):
#     s += " USA"  # //Append USA
#     cleanedBirthplace = s
#     print(cleanedBirthplace)
# #//Append country if last element length is 2
# elif (len(strArray[len(strArray)-1])-1) == 2:
#     s += " USA"
#     cleanedBirthplace = "".join(s)
#     print(cleanedBirthplace)
# else:
#     cleanedBirthplace = s
#     print(cleanedBirthplace)

In [ ]:
# print(s)
# print(len(s))
# print(s[len(s)-1])
# print(len(s[len(s)-1]))
# print("")

# print(strArray)
# print(len(strArray))
# print(len(strArray[len(strArray)-1]))
# print(len(strArray[len(strArray)-1])-1)
